In [1]:
from graphframes import *

In [2]:
trip = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("file:///home/cloudera/trip.csv")

In [3]:
station = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("file:///home/cloudera/station.csv")

In [4]:
trip.registerTempTable("trip")
trip.cache()

DataFrame[Trip ID: int, Duration: int, Start Date: string, Start Station: string, Start Terminal: int, End Date: string, End Station: string, End Terminal: int, Bike #: int, Subscriber Type: string, Zip Code: string]

In [5]:
station.registerTempTable("station")
station.cache()

DataFrame[station_id: int, name: string, lat: double, long: double, dockcount: int, landmark: string, installation: string]

In [6]:
sqlContext.sql("SELECT * from station").show(4)

+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 4 rows



In [10]:
sqlContext.sql("SELECT * from trip").show(4)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|Start Terminal|       End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

In [7]:
st_unique=sqlContext.sql("select distinct name,lat,long from station")
st_unique.registerTempTable("st_unique")

from pyspark.sql.functions import *
from graphframes import *

vertices = st_unique.withColumnRenamed("name", "id").distinct()

trip = trip.withColumnRenamed("Start Station","src")

trip = trip.withColumnRenamed("End Station","dst")

tripEdges = trip.select('Trip ID', 'Duration','src', 'dst','Start Terminal','End Terminal','Start Date','End Date','BIke #')

tripEdges.cache()
vertices.cache()

vertices.show(4)

tripEdges.show(4)


g = GraphFrame(vertices,tripEdges)
print g

g.vertices.show()

g.edges.show()

inDg = g.inDegrees
inDg.show(10)

out_Dg =g.outDegrees
out_Dg.show(10)

g.degrees.show()

g.inDegrees.count()

g.outDegrees.count()

g.inDegrees.sort(desc("inDegree")).show(10)

g.outDegrees.sort(desc("outDegree")).show(10)

motifs = g.find("(a)-[e]->(b)")
motifs.show()

motifs.e.count()


import pyspark.sql.functions as func
topTrips = g.edges.groupBy("src", "dst").agg(func.count("Trip ID").alias("trips"))


topTrips.orderBy(topTrips.trips.desc()).limit(20).show()


#PageRank algorithm
results = g.pageRank(resetProbability=0.15, maxIter = 2)

results.vertices.orderBy(results.vertices.pagerank.desc()).limit(20).show()

motifs = g.find("(a)-[ab]->(b); (b)-[bc]->(c)")
motifs.show()

###outdegrees/indegrees
dg_Ratio = inDg.join(out_Dg,inDg.id == out_Dg.id).drop(out_Dg.id)
dg_Ratio.show()

In [7]:
st_unique=sqlContext.sql("select distinct name,lat,long from station")
st_unique.registerTempTable("st_unique")

In [8]:
from pyspark.sql.functions import *
from graphframes import *

vertices = st_unique.withColumnRenamed("name", "id").distinct()

trip = trip.withColumnRenamed("Start Station","src")

trip = trip.withColumnRenamed("End Station","dst")

tripEdges = trip.select('Trip ID', 'Duration','src', 'dst','Start Terminal','End Terminal','Start Date','End Date','BIke #')


In [9]:
tripEdges.cache()
vertices.cache()
vertices.show(4)
tripEdges.show(4)

+--------------------+---------+-----------+
|                  id|      lat|       long|
+--------------------+---------+-----------+
|       2nd at Folsom|37.785299|-122.396236|
|   2nd at South Park|37.782259|-122.392738|
|Rengstorff Avenue...|37.400241|-122.099076|
|California Ave Ca...|37.429082|-122.142805|
+--------------------+---------+-----------+
only showing top 4 rows

+-------+--------+--------------------+--------------------+--------------+------------+---------------+---------------+------+
|Trip ID|Duration|                 src|                 dst|Start Terminal|End Terminal|     Start Date|       End Date|BIke #|
+-------+--------+--------------------+--------------------+--------------+------------+---------------+---------------+------+
| 913460|     765|Harry Bridges Pla...|San Francisco Cal...|            50|          70|8/31/2015 23:26|8/31/2015 23:39|   288|
| 913459|    1036|San Antonio Shopp...|Mountain View Cit...|            31|          27|8/31/2015 23:11

In [10]:
g = GraphFrame(vertices,tripEdges)
print g

g.vertices.show()

g.edges.show()


GraphFrame(v:[id: string, lat: double, long: double], e:[src: string, dst: string, Trip ID: int, Duration: int, Start Terminal: int, End Terminal: int, Start Date: string, End Date: string, BIke #: int])
+--------------------+---------+-----------+
|                  id|      lat|       long|
+--------------------+---------+-----------+
|       2nd at Folsom|37.785299|-122.396236|
|   2nd at South Park|37.782259|-122.392738|
|Rengstorff Avenue...|37.400241|-122.099076|
|California Ave Ca...|37.429082|-122.142805|
|Cowper at University|37.448598|-122.159504|
|Harry Bridges Pla...|37.795392|-122.394203|
|         Ryland Park|37.342725|-121.895617|
|  San Jose City Hall|37.337391|-121.886995|
|Embarcadero at Br...|37.787152|-122.388013|
|       5th at Howard|37.781752|-122.405127|
|SJSU - San Salvad...|37.333955|-121.877349|
|Commercial at Mon...|37.794231|-122.402923|
|Evelyn Park and Ride|37.390277|-122.066553|
|     Clay at Battery|37.795001| -122.39997|
|San Antonio Caltr...| 37.40694

In [19]:
inDg = g.inDegrees
inDg.show(10)

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|Embarcadero at Br...|    6687|
|   Market at Sansome|   13916|
|       Park at Olive|     417|
|Washington at Kearny|    3481|
|Evelyn Park and Ride|     725|
|    San Pedro Square|    1595|
|San Antonio Caltr...|    1046|
|     2nd at Townsend|   15463|
| San Salvador at 1st|     547|
|         MLK Library|     960|
+--------------------+--------+
only showing top 10 rows



In [20]:
out_Dg =g.outDegrees
out_Dg.show(10)

+--------------------+---------+
|                  id|outDegree|
+--------------------+---------+
|Embarcadero at Br...|     7483|
|   Market at Sansome|    11431|
|       Park at Olive|      376|
|Washington at Kearny|     2660|
|Evelyn Park and Ride|      978|
|    San Pedro Square|     1418|
|San Antonio Caltr...|     1058|
|     2nd at Townsend|    14026|
|         MLK Library|     1099|
| San Salvador at 1st|      495|
+--------------------+---------+
only showing top 10 rows



In [ ]:
g.degrees.show()

g.inDegrees.count()

g.outDegrees.count()

In [11]:
g.inDegrees.sort(desc("inDegree")).show(10)


+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|   34810|
|San Francisco Cal...|   22523|
|Harry Bridges Pla...|   17810|
|     2nd at Townsend|   15463|
|     Townsend at 7th|   15422|
|Embarcadero at Sa...|   15065|
|   Market at Sansome|   13916|
|   Steuart at Market|   13617|
|Temporary Transba...|   12966|
|  Powell Street BART|   10239|
+--------------------+--------+
only showing top 10 rows



In [12]:
g.outDegrees.sort(desc("outDegree")).show(10)

+--------------------+---------+
|                  id|outDegree|
+--------------------+---------+
|San Francisco Cal...|    26304|
|San Francisco Cal...|    21758|
|Harry Bridges Pla...|    17255|
|Temporary Transba...|    14436|
|Embarcadero at Sa...|    14158|
|     2nd at Townsend|    14026|
|     Townsend at 7th|    13752|
|   Steuart at Market|    13687|
|      Market at 10th|    11885|
|   Market at Sansome|    11431|
+--------------------+---------+
only showing top 10 rows



In [15]:
motifs = g.find("(a)-[e]->(b)")
motifs.show(10)

motifs.count()

+--------------------+--------------------+--------------------+
|                   e|                   a|                   b|
+--------------------+--------------------+--------------------+
|[913460,765,Harry...|[Harry Bridges Pl...|[San Francisco Ca...|
|[913459,1036,San ...|[San Antonio Shop...|[Mountain View Ci...|
|[913454,409,San J...|[San Jose City Ha...|[San Salvador at ...|
|[913453,789,Embar...|[Embarcadero at F...|[Embarcadero at S...|
|[913452,293,Yerba...|[Yerba Buena Cent...|[San Francisco Ca...|
|[913451,896,Embar...|[Embarcadero at F...|[Embarcadero at S...|
|[913450,255,Embar...|[Embarcadero at S...|[Steuart at Marke...|
|[913449,126,Beale...|[Beale at Market,...|[Temporary Transb...|
|[913443,691,Embar...|[Embarcadero at S...|[Market at Sansom...|
|[913442,633,Marke...|[Market at 10th,3...|[San Francisco Ca...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



339030

In [ ]:
import pyspark.sql.functions as func
topTrips = g.edges.groupBy("src", "dst").agg(func.count("Trip ID").alias("trips"))

In [ ]:
topTrips.orderBy(topTrips.trips.desc()).limit(20).show()

In [16]:

motifs = g.find("(a)-[ab]->(b); (b)-[bc]->(c)")
motifs.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  ab|                   a|                   b|                  bc|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[913415,274,Harry...|[Harry Bridges Pl...|[Embarcadero at B...|[913386,1808,Emba...|[Harry Bridges Pl...|
|[913415,274,Harry...|[Harry Bridges Pl...|[Embarcadero at B...|[913204,453,Embar...|[San Francisco Ca...|
|[913415,274,Harry...|[Harry Bridges Pl...|[Embarcadero at B...|[912934,366,Embar...|[San Francisco Ca...|
|[913415,274,Harry...|[Harry Bridges Pl...|[Embarcadero at B...|[912823,284,Embar...|[Harry Bridges Pl...|
|[913415,274,Harry...|[Harry Bridges Pl...|[Embarcadero at B...|[912653,317,Embar...|[San Francisco Ca...|
|[913415,274,Harry...|[Harry Bridges Pl...|[Embarcadero at B...|[912472,642,Embar...|[Embarcadero at S...|
|[913415,274,Harry...|[Harry Bridges 

In [17]:
#PageRank algorithm
results = g.pageRank(resetProbability=0.15, maxIter = 2)

results.vertices.orderBy(results.vertices.pagerank.desc()).limit(20).show()

+--------------------+---------+-----------+-------------------+
|                  id|      lat|       long|           pagerank|
+--------------------+---------+-----------+-------------------+
|San Jose Diridon ...|37.329732|-121.901782|  1.214473817911346|
|San Francisco Cal...|37.776617| -122.39526| 1.0496950452048421|
|Redwood City Calt...|37.486078|-122.232089| 0.8006700627240692|
|Mountain View Cal...|37.394358|-122.076713|  0.753839953327208|
|San Francisco Cal...|  37.7766| -122.39547|  0.726896036435997|
|Harry Bridges Pla...|37.795392|-122.394203| 0.5768070317865084|
|     Townsend at 7th|37.771058|-122.402717| 0.5223761109233136|
|     2nd at Townsend|37.780526|-122.390288| 0.5196239311404478|
|Embarcadero at Sa...| 37.80477|-122.403234| 0.5166722126882114|
|Palo Alto Caltrai...|37.443988|-122.164759| 0.5102959418842666|
|   Market at Sansome|37.789625|-122.400811| 0.4985964480246575|
|Santa Clara at Al...|37.333988|-121.894902| 0.4955425352860525|
|   Steuart at Market|37.